In [1]:
# David Ouyang 
# Ishan Jain 
# 9/25/2020
# Training loop to choose ESV/EDV frame count

import statistics
import echonet
import os
import collections
import numpy as np
import tqdm
import math 
import skimage.segmentation
import scipy
import torch
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import re
import PIL.Image
import cv2
import random

print(echonet.__file__)

#outputpath = "C:\\Users\\Remote\\Documents\\ishanjain\\echonet-function-evaluation\\Deep Learning\\output\\video\\r2plus1d_18_32_1_pretrained"
outputpath = "/Users/ishan/Documents/Stanford/EchoData/output/video/r2plus1d_18_32_1_pretrained"

/Users/ishan/Documents/Stanford/echonet-function-evaluation/Deep Learning/echonet/__init__.py


In [2]:
#Test 

mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="test"))
print(mean,std)

modelname="r2plus1d_18"
frames=32
period=1
tasks="NormalizedLargeIndex"#"NormalizedSmallIndex"
pretrained=True

kwargs = {"target_type": tasks,
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period
          }

    # Set up datasets and dataloaders
test_dataset = echonet.datasets.Echo(split="test", **kwargs)

print(len(test_dataset))

100%|██████████| 16/16 [00:17<00:00,  1.07s/it]


[33.774605 33.955395 34.318237] [51.00695 51.19394 51.51023]
1276


In [3]:
numWork = 5
batch_size=8

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

dataloaders = {'test':test_dataloader}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set up model
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

# for phase in dataloaders.keys():
#     with torch.set_grad_enabled(phase == "train"):
#         with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
#             for i, (x, outcome) in enumerate(dataloaders[phase]):
#                 print(i, x.shape, outcome)


In [4]:
#this is for large index

num_epochs = 1
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint = os.path.join(outputpath, "verbose_normalizedlargeindex_best.pt") # you need to load different weights

toGPU = torch.cuda.is_available()
#toGPU = False

model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

if toGPU:
    print("CUDA is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("Loading Weights")
        checkpoint = torch.load(priorCheckpoint)
        
        model.load_state_dict(checkpoint['state_dict'])
        model.eval()
        
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
        
logFilename = os.path.join(outputpath, "test_normalizedlargeindex.csv")

totaldiff = 0
differences = []

with open(logFilename, 'w') as output:
    for epoch in range(1, 2):
        for phase in dataloaders.keys():

            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    print(enumerate(dataloaders[phase]))
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        
                        #print(x, outcome)
                        x = x.to(device)
                        outcome = outcome.to(device)
                        p = model(x)#["out"]
                        #output.write(#Filename, Prediction, GroundTruth, Difference betwen the two)
                        
                        for i in range(batch_size):
                            output.write(str(i) + "," + str(round(p[i].item())) + "," + str(outcome[i].item()) + "," + str( abs(round(p[i].item()) - outcome[i].item())) + "\n")
                            #print(str(round(p[i].item())) + "," + str(outcome[i].item()) + "," + str( abs(round(p[i].item()) - outcome[i].item())))
                            totaldiff += abs(round(p[i].item()) - outcome[i].item())
                            differences.append(abs(round(p[i].item()) - outcome[i].item()))
                        #loss = torch.nn.functional.mse_loss(p.view(-1), outcome)

                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f}".format(epoch, i))
                        progressbar.update()
                        
                        print("This is the average difference:", totaldiff / 1276)
print(statistics.pstdev(differences))

cuda is not available, cpu weights


  0%|          | 0/159 [00:00<?, ?it/s]

  0%|          | 0/159 [00:14<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'

In [15]:
#this is for large index

num_epochs = 1
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint = os.path.join(outputpath, "verbose_normalizedsmallindex_best2.pt") # you need to load different weights

toGPU = torch.cuda.is_available()
#toGPU = False


model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)


if toGPU:
    print("CUDA is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("Loading Weights")
        checkpoint = torch.load(priorCheckpoint)
        
        
        model.load_state_dict(checkpoint['state_dict'])
        model.eval()
        
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
        
        
logFilename = os.path.join(outputpath, "scripts", "test_normalizedsmallindex.csv")

totaldiff = 0

with open(logFilename, 'w') as output:
    for epoch in range(1, 2):
        for phase in dataloaders.keys():

            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        
                        #print(x, outcome)
                        x = x.to(device)
                        outcome = outcome.to(device)
                        p = model(x)#["out"]
                        #output.write(#Filename, Prediction, GroundTruth, Difference betwen the two)
                        
                        for i in range(batch_size):
                            output.write(str(round(p[i].item())) + "," + str(outcome[i].item()) + "," + str( abs(round(p[i].item()) - outcome[i].item())) + "\n")
                            #print(str(round(p[i].item())) + "," + str(outcome[i].item()) + "," + str( abs(round(p[i].item()) - outcome[i].item())))
                            totaldiff += abs(round(p[i].item()) - outcome[i].item())
                            std.append()
                        #loss = torch.nn.functional.mse_loss(p.view(-1), outcome)

                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f}".format(epoch, i))
                        progressbar.update()
                        
                        print("This is the average difference:", totaldiff / 1276)

CUDA is available
Loading Weights


  1%|▊                                                                 | 2/159 [00:13<24:59,  9.55s/it, test1.0 7.0000]

This is the average difference: 0.0603448275862069
This is the average difference: 0.10266457680250783


  3%|█▋                                                                | 4/159 [00:14<12:18,  4.76s/it, test1.0 7.0000]

This is the average difference: 0.14655172413793102
This is the average difference: 0.1896551724137931


...

 99%|███████████████████████████████████████████████████████████████▌| 158/159 [00:38<00:00,  6.38it/s, test1.0 7.0000]

This is the average difference: 7.962382445141066
This is the average difference: 8.0141065830721


100%|████████████████████████████████████████████████████████████████| 159/159 [00:38<00:00,  6.35it/s, test1.0 7.0000]

This is the average difference: 8.061128526645769


100%|████████████████████████████████████████████████████████████████| 159/159 [00:38<00:00,  4.12it/s, test1.0 7.0000]


In [39]:
import random

totaldiff = 0

for i in range(10000):
    #print(random.randint(0,32))
    totaldiff += abs(random.randint(0,32) - random.randint(0,32))
    
print(totaldiff / 10000)

10.9694


In [35]:
# This is for large index


In [6]:
mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="train"))
print(mean,std)

modelname="r2plus1d_18"
frames=32
period=1
tasks="NormalizedLargeIndex"#"NormalizedSmallIndex"
pretrained=True 


kwargs = {"target_type": tasks,
              "mean": mean,
              "std": std,
              "length": frames,
              "period": period,
              }

    # Set up datasets and dataloaders
train_dataset = echonet.datasets.Echo(split="train", **kwargs)

val_dataset = echonet.datasets.Echo(split="val", **kwargs)

print(len(train_dataset), len(val_dataset))

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:11<00:00,  1.36it/s]


[33.08518  33.43475  34.045193] [49.781414 50.02839  50.6773  ]
7460 1288


In [7]:
numWork = 5
batch_size=8

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set up model
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

# for phase in dataloaders.keys():
#     with torch.set_grad_enabled(phase == "train"):
#         with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
#             for i, (x, outcome) in enumerate(dataloaders[phase]):
#                 print(i, x.shape, outcome)


In [4]:
#this is for large index

num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_normalizedlargeindex_checkpoint.pt")

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_normalizedlargeindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    if not os.path.exists(os.path.join(outputpath, "scripts")):
        os.mkdir(os.path.join(outputpath, "scripts"))

with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):

                        x = x.to(device)
                        outcome = outcome.to(device)
                        p = model(x)#["out"]

                        loss = torch.nn.functional.mse_loss(p.view(-1), outcome)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_normalizedlargeindex_checkpoint.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_normalizedlargeindex_best.pt"))
                bestLoss = total / n
                print("This is the best yet")


cuda is available


  0%|                                                    | 1/932 [00:22<5:49:15, 22.51s/it, train1.0 39.2122 (39.2122)]C:\Users\Remote\AppData\Roaming\Python\Python38\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


100%|███████████████████████████████████████████████████████| 161/161 [00:39<00:00,  4.10it/s, val44.0 9.5673 (8.5705)]


In [44]:
mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="train"))
print(mean,std)

modelname="r3d_18"
frames=32
period=1
tasks="NormalizedLargeIndex"#"NormalizedSmallIndex"
pretrained=True 


kwargs = {"target_type": tasks,
              "mean": mean,
              "std": std,
              "length": frames,
              "period": period,
              }

    # Set up datasets and dataloaders
train_dataset = echonet.datasets.Echo(split="train", **kwargs)

val_dataset = echonet.datasets.Echo(split="val", **kwargs)

print(len(train_dataset), len(val_dataset))

numWork = 5
batch_size=8

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set up model
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

# for phase in dataloaders.keys():
#     with torch.set_grad_enabled(phase == "train"):
#         with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
#             for i, (x, outcome) in enumerate(dataloaders[phase]):
#                 print(i, x.shape, outcome)


#this is for large index

num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_r3d_normalizedlargeindex_checkpoint.pt")

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_r3d_normalizedlargeindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    if not os.path.exists(os.path.join(outputpath, "scripts")):
        os.mkdir(os.path.join(outputpath, "scripts"))
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
#                         if verbose:
#                             print(i, x.shape, outcome)
                        x = x.to(device)
                        outcome = outcome.to(device)
                        p = model(x)#["out"]
#                         if verbose:
#                             print( x.shape, p.shape, outcome.shape)
                            
                        loss = torch.nn.functional.mse_loss(p.view(-1), outcome)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_r3d_normalizedlargeindex_checkpoint.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_r3d_normalizedlargeindex_best.pt"))
                bestLoss = total / n
                print("This is the best yet")


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:13<00:00,  1.15it/s]


[33.415447 33.460274 33.60476 ] [50.284832 50.356934 50.523277]
7460 1288


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

cuda is available


  0%|                                                    | 1/932 [00:13<3:29:44, 13.52s/it, train1.0 36.9018 (36.9018)]C:\Users\Remote\AppData\Roaming\Python\Python38\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


100%|███████████████████████████████████████████████████████| 161/161 [00:39<00:00,  4.11it/s, val44.0 9.3188 (5.3197)]


In [8]:

# This is for large index
# Classification task instead of regression task on frame number

# Changed model, with num classes as number of frames, and changed into cross entropy task and one_hot encoding of classes


model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = frames)
#model.fc = torch.nn.Linear(model.fc.in_features, 1)



num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_classification_normalizedlargeindex_checkpoint.pt")
print(priorCheckpoint, os.path.exists(priorCheckpoint))

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_classification_normalizedlargeindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    if not os.path.exists( os.path.join(outputpath, "scripts")):
        os.mkdir(os.path.join(outputpath, "scripts"))
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        
    
#                         if verbose:
#                             print(i, outcome, outcome.type(torch.int64))
                        
#                         #one hot encoding of outcome, also needs to be int instead of double to use one_hot
                        
#                         one_hot = torch.nn.functional.one_hot(outcome.type(torch.int64), num_classes = frames)

#                         if verbose:
#                             print(i, x.shape, outcome.shape, one_hot.shape)
                        
                        x = x.to(device)
                        outcome = outcome.to(device)
                        #one_hot = one_hot.to(device)
                        p = model(x)#["out"]
#                         if verbose:
#                             print( x.shape, p.shape, outcome.shape)
                        
    
                        loss = torch.nn.functional.cross_entropy(p, outcome.type(torch.int64)) #one_hot)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedlargeindex_checkpoint.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedlargeindex_best.pt"))
                bestLoss = total / n
                print("This is the best yet")

                

  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

C:\Users\Remote\Documents\ishanjain\echonet-function-evaluation\Deep Learning\output\video\r2plus1d_18_32_1_pretrained\verbose_classification_normalizedlargeindex_checkpoint.pt False
cuda is available


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


100%|███████████████████████████████████████████████████████| 161/161 [00:39<00:00,  4.11it/s, val44.0 0.4288 (0.4253)]


In [ ]:











# This is for large index
# Classification task instead of regression task on frame number

# Changed model, with num classes as number of frames, and changed into cross entropy task and one_hot encoding of classes


model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = frames)
#model.fc = torch.nn.Linear(model.fc.in_features, 1)



num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_classification_normalizedlargeindex_checkpoint.pt")
print(priorCheckpoint, os.path.exists(priorCheckpoint))

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_classification_normalizedlargeindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    if not os.path.exists( os.path.join(outputpath, "scripts")):
        os.mkdir(os.path.join(outputpath, "scripts"))
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        
    
#                         if verbose:
#                             print(i, outcome, outcome.type(torch.int64))
                        
#                         #one hot encoding of outcome, also needs to be int instead of double to use one_hot
                        
#                         one_hot = torch.nn.functional.one_hot(outcome.type(torch.int64), num_classes = frames)

#                         if verbose:
#                             print(i, x.shape, outcome.shape, one_hot.shape)
                        
                        x = x.to(device)
                        outcome = outcome.to(device)
                        #one_hot = one_hot.to(device)
                        p = model(x)#["out"]
#                         if verbose:
#                             print( x.shape, p.shape, outcome.shape)
                        
    
                        loss = torch.nn.functional.cross_entropy(p, outcome.type(torch.int64)) #one_hot)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedlargeindex_checkpoint.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedlargeindex_best.pt"))
                bestLoss = total / n
                print("This is the best yet")

                






In [1]:
# This is for small index


In [2]:
mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="train"))
print(mean,std)

modelname="r2plus1d_18"
frames=32
period=1
tasks="NormalizedSmallIndex"
pretrained=True 


kwargs = {"target_type": tasks,
              "mean": mean,
              "std": std,
              "length": frames,
              "period": period,
              }

    # Set up datasets and dataloaders
train_dataset = echonet.datasets.Echo(split="train", **kwargs)

val_dataset = echonet.datasets.Echo(split="val", **kwargs)

print(len(train_dataset), len(val_dataset))

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.33it/s]


[32.148865 32.46407  32.90127 ] [49.9002   50.190792 50.71088 ]
7460 1288


In [3]:
numWork = 5
batch_size=8

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set up model
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)

# for phase in dataloaders.keys():
#     with torch.set_grad_enabled(phase == "train"):
#         with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
#             for i, (x, outcome) in enumerate(dataloaders[phase]):
#                 print(i, x.shape, outcome)

In [6]:
# This is for small index



num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_normalizedsmallindex_checkpoint.pt")
priorCheckpoint = "NA"
print(priorCheckpoint, os.path.exists(priorCheckpoint))

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_normalizedsmallindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    os.mkdir(os.path.join(outputpath, "scripts"))
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
#                         if verbose:
#                             print(i, x.shape, outcome.shape)
                        x = x.to(device)
                        outcome = outcome.to(device)
                        p = model(x)#["out"]
#                         if verbose:
#                             print( x.shape, p.shape, outcome.shape)
                            
                        loss = torch.nn.functional.mse_loss(p.view(-1), outcome)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_normalizedsmallindex_checkpoint2.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_normalizedsmallindex_best2.pt"))
                bestLoss = total / n
                print("This is the best yet")


NA False
cuda is available


  0%|                                                    | 1/932 [00:18<4:45:47, 18.42s/it, train1.0 54.2464 (54.2464)]C:\Users\Remote\AppData\Roaming\Python\Python38\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


100%|██████████████████████████████████████████████████████| 161/161 [00:39<00:00,  4.09it/s, val44.0 10.7642 (8.4007)]


#### FIRST RUN #### HERE are the numbers for the first 14 epochs in first run


cuda is available
  0%|                                                    | 1/932 [00:15<4:06:40, 15.90s/it, train1.0 55.1535 (55.1535)]C:\Users\Remote\AppData\Roaming\Python\Python38\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|████████████████████████████████████████████████████| 932/932 [10:36<00:00,  1.46it/s, train1.0 12.3963 (15.8264)]
100%|██████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.36it/s, val1.0 11.3351 (14.4505)]
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]
This is the best yet
100%|████████████████████████████████████████████████████| 932/932 [10:31<00:00,  1.48it/s, train2.0 11.1962 (10.4629)]
100%|██████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.38it/s, val2.0 10.9464 (11.3590)]
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]
This is the best yet
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train3.0 11.1868 (15.0299)]
100%|███████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.38it/s, val3.0 10.6582 (8.5865)]
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]
This is the best yet
100%|█████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train4.0 11.2358 (8.3774)]
100%|██████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.38it/s, val4.0 10.6469 (15.1854)]
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]
This is the best yet
100%|█████████████████████████████████████████████████████| 932/932 [10:31<00:00,  1.48it/s, train5.0 11.2808 (8.0343)]
100%|███████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.36it/s, val5.0 10.7765 (9.4355)]
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train6.0 11.2152 (13.3924)]
100%|██████████████████████████████████████████████████████| 161/161 [00:36<00:00,  4.40it/s, val6.0 10.8254 (14.5315)]
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train7.0 11.1704 (11.8043)]
100%|███████████████████████████████████████████████████████| 161/161 [00:38<00:00,  4.21it/s, val7.0 10.9360 (5.3713)]
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train8.0 11.2137 (11.0053)]
100%|███████████████████████████████████████████████████████| 161/161 [00:37<00:00,  4.35it/s, val8.0 10.2768 (6.6230)]
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]
This is the best yet
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train9.0 11.3659 (10.9468)]
100%|███████████████████████████████████████████████████████| 161/161 [00:37<00:00,  4.33it/s, val9.0 10.7838 (6.2929)]
100%|███████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train10.0 11.3453 (11.4390)]
100%|█████████████████████████████████████████████████████| 161/161 [00:37<00:00,  4.31it/s, val10.0 10.8094 (11.4828)]
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train11.0 11.1566 (9.6238)]
100%|█████████████████████████████████████████████████████| 161/161 [00:37<00:00,  4.30it/s, val11.0 10.5223 (11.2068)]
100%|████████████████████████████████████████████████████| 932/932 [10:32<00:00,  1.47it/s, train12.0 10.9960 (8.4571)]
100%|██████████████████████████████████████████████████████| 161/161 [00:37<00:00,  4.32it/s, val12.0 11.0499 (8.3510)]
100%|███████████████████████████████████████████████████| 932/932 [10:34<00:00,  1.47it/s, train13.0 11.2850 (11.6567)]
100%|██████████████████████████████████████████████████████| 161/161 [01:38<00:00,  1.63it/s, val13.0 10.9843 (9.3447)]

In [5]:

# This is for small index
# Classification task instead of regression task on frame number

# Changed model, with num classes as number of frames, and changed into cross entropy task and one_hot encoding of classes


model = torchvision.models.video.r2plus1d_18(pretrained=False, num_classes = frames)
#model.fc = torch.nn.Linear(model.fc.in_features, 1)



num_epochs = 45
verbose = True

startingEpoch = 0
startingBestLoss = 100000
priorCheckpoint =  os.path.join(outputpath, "verbose_classification_normalizedsmallindex_checkpoint.pt")
print(priorCheckpoint, os.path.exists(priorCheckpoint))

toGPU = torch.cuda.is_available()
#toGPU = False
if toGPU:
    print("cuda is available")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
if os.path.exists(priorCheckpoint):   
    if toGPU:
        print("loading weights")
        checkpoint = torch.load(priorCheckpoint)
        model.load_state_dict(checkpoint['state_dict'])
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
    else:
        print("cuda is not available, cpu weights")
        device = torch.device("cpu")
        checkpoint = torch.load(priorCheckpoint, map_location = "cpu")
        state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
        model.load_state_dict(state_dict_cpu)
        startingEpoch = checkpoint['epoch']
        startingBestLoss = checkpoint['best_loss']
learning_rate = 1e-4
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
bestLoss = startingBestLoss
priorLines = None
logFilename = os.path.join(outputpath, "scripts", "verbose_classification_normalizedsmallindex.csv")
if os.path.exists(logFilename):
    with open(logFilename, 'r') as output:
        priorLines = output.readlines()
else:
    if not os.path.exists( os.path.join(outputpath, "scripts")):
        os.mkdir(os.path.join(outputpath, "scripts"))
# with open(logFilename, 'w') as output:
#     for line in priorLines:
#         output.write(line)
with open(logFilename, 'w') as output:
    for epoch in range(startingEpoch + 1, num_epochs):
        if priorLines is not None:
            for line in priorLines:
                output.write(line)
                print(line)
        for phase in dataloaders.keys():
            total = 0
            n = 0
            with torch.set_grad_enabled(phase == "train"):
                with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                    for i, (x, outcome) in enumerate(dataloaders[phase]):
                        
    
#                         if verbose:
#                             print(i, outcome, outcome.type(torch.int64))
                        
#                         #one hot encoding of outcome, also needs to be int instead of double to use one_hot
                        
#                         one_hot = torch.nn.functional.one_hot(outcome.type(torch.int64), num_classes = frames)

#                         if verbose:
#                             print(i, x.shape, outcome.shape, one_hot.shape)
                        
                        x = x.to(device)
                        outcome = outcome.to(device)
                        #one_hot = one_hot.to(device)
                        p = model(x)#["out"]
#                         if verbose:
#                             print( x.shape, p.shape, outcome.shape)
                        
    
                        loss = torch.nn.functional.cross_entropy(p, outcome.type(torch.int64)) #one_hot)
                        
                        total += loss.item()
                        n += x.shape[0]
                        progressbar.set_postfix_str(phase +"{:.1f} {:.4f} ({:.4f})".format(epoch, total / n, loss.item() /  x.shape[0]))
                        progressbar.update()
                        if(phase == 'train'):
                            optim.zero_grad()
                            loss.backward()
                            optim.step()
            output.write(phase + "," + str(epoch) + "," + str(total/n) + "\n")
            output.flush()
            save = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'period': period,
                'frames':frames,
                'best_loss': bestLoss,
                'loss': total / n,
                'opt_dict': optim.state_dict()
                }
            torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedsmallindex_checkpoint2.pt"))
            if total / n < bestLoss and phase == "val":
                torch.save(save, os.path.join(outputpath, "verbose_classification_normalizedsmallindex_best2.pt"))
                bestLoss = total / n
                print("This is the best yet")

                

  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

C:\Users\Remote\Documents\ishanjain\echonet-function-evaluation\Deep Learning\output\video\r2plus1d_18_32_1_pretrained\verbose_classification_normalizedsmallindex_checkpoint.pt False
cuda is available


  0%|                                                      | 1/932 [00:13<3:25:18, 13.23s/it, train1.0 0.4486 (0.4486)]C:\Users\Remote\AppData\Roaming\Python\Python38\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


  0%|                                                                                          | 0/932 [00:00<?, ?it/s]

This is the best yet


100%|███████████████████████████████████████████████████████| 161/161 [00:38<00:00,  4.16it/s, val44.0 0.4306 (0.4332)]


In [20]:
n_classes = 32
n_samples = 10

# Create list n_samples random labels (can also be numpy array)
labels = [random.randrange(n_classes) for _ in range(n_samples)]
# Convert to torch Tensor
labels_tensor = torch.as_tensor(labels)


# Create one-hot encodings of labels
one_hot = torch.nn.functional.one_hot(labels_tensor, num_classes=n_classes)
print(labels_tensor.shape, one_hot.shape)

torch.Size([10]) torch.Size([10, 32])


In [21]:
print(outcome, outcome.type(torch.int8), labels_tensor)

tensor([21.,  4., 24.,  8., 27.,  1., 17.,  7.]) tensor([21,  4, 24,  8, 27,  1, 17,  7], dtype=torch.int8) tensor([26, 13, 14,  0, 11, 17,  9, 30, 13, 12])


In [24]:
one_hot = torch.nn.functional.one_hot(labels_tensor, num_classes=n_classes)
print(one_hot)

RuntimeError: one_hot is only applicable to index tensor.

In [25]:
one_hot = torch.nn.functional.one_hot(outcome.type(torch.int64), num_classes=n_classes)
print(one_hot)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
# constrainedloss testing
# search just training on CE

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

numWork = 10
numberOfSubsequentFrames = 5
bs = 45
numEpochs = 50

for weightOfConstraint in [0]: #[0.8, 0.85, 0.9, 0.95]: #  0.5, 0.1, 0, 0.9, 1 ]: #1, 0,  0.9, #   1000, 100,  10, 1]:
    for CrossEntropyLoss in [1]:
        for SonoData in [False]:
            for doAugmentation in [False]:
                for doPretrain in [True]:


                    train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)

                    if SonoData:
                        sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=True)
                        train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)
                    else:
                        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)
                    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                    device = torch.device("cuda")

                    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                    model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                    if device.type == "cuda":
                        model = torch.nn.DataParallel(model)
                    model.to(device)

                    if doPretrain:
                        print("load from weights")
                        checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                        model.load_state_dict(checkpoint['state_dict'])

                    #optim = torch.optim.SGD(model.parameters(), lr=1000, momentum=0.9)
                    learning_rate = 1e-2
                    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    #optim = torch.optim.Adagrad(model.parameters(), lr=100)

                    bestLoss = 1000000

                    with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_withCrossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint) + "_"+str(numEpochs)+"_Adam.csv", 'w') as output:
                        for epoch in range(numEpochs):
                            for phase in dataloaders.keys():

                                total = 0
                                n = 0
                                dsctotal = 0
                                cetotal = 0
                                constrainedTotal = 0

                                with torch.set_grad_enabled(phase != 'val'):
                                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                        for (i, (x,y)) in enumerate(dataloaders[phase]):

                                            x = x.to(device)
                                            y = y.to(device)
                                            #print(y.min(), y.max())
                                            #print(x.shape)

                                            #print("input shape", x.shape)

                                            # reshape to be bs x frames indepedent predictions on (3,112,112) images
                                            x = torch.transpose(x, 1, 4)
                                            #print(x.shape)
                                            x = x.reshape(bs * numberOfSubsequentFrames,  112, 112, 3).transpose(1,2)
                                            x = torch.transpose(x, 1, 3).type(torch.float)

                                            #print("new input shape", x.shape)

                                            p = model(x)["out"]

                                            #print("predictions shape", p.shape)

                                            constrainedLoss = variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames)
                                            dscloss = x.shape[0] / numberOfSubsequentFrames - variousLossFunctions.diceLoss(p[0::numberOfSubsequentFrames, 0, :, :], y,)
                                            celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsequentFrames, 0, :, :], y, reduction = "sum") / 112 / 112

                                            loss =  (1-weightOfConstraint) * celoss + weightOfConstraint * constrainedLoss
                                            #CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsquentFrames, 0, :, :], y, reduction = "sum") / 112 / 112 + variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames).item()#x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,) +
                                            
#                                            loss = loss.item()
                                            total += loss#.item()
                                            dsctotal += dscloss#.item()
                                            cetotal += celoss#.item()
                                            constrainedTotal += constrainedLoss

                                            n += x.shape[0] / numberOfSubsequentFrames

                                            #print(loss, total, n, flush = True)

                                            progressbar.set_postfix_str(phase + "{:.1f} total{:.4f} dsc{:.4f} ce{:.4f} constrain{:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , constrainedTotal / n, loss / x.size(0) * numberOfSubsequentFrames )) #/ 112 / 112
                                            progressbar.update()

                                            if(phase != 'val'):
                                                optim.zero_grad()
                                                loss.backward()
                                                optim.step()

                                        output.write(phase + "," + str(epoch) + "," + str(total.item() / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + ',' + str(constrainedTotal.item() / n) + "\n") #/ 112 / 112

                                        if phase == 'val':
                                            currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_current.pt"
                                            torch.save(model.state_dict(), currentRVweights)

                                            if (total / n ) < bestLoss: #/ 112 / 112)
                                                bestLoss = (total / n ) # / 112 / 112)
                                                print("this is the best so far", str(bestLoss), flush = True)
                                                bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) +"_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_best.pt"
                                                torch.save(model.state_dict(), bestRVweights)


In [ ]:
# constrainedloss testing with smaller loss, same sweep region

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

numWork = 10
numberOfSubsequentFrames = 5
bs = 45
numEpochs = 100

for weightOfConstraint in  [0.8, 0.85, 0.9, 0.95]: #[0, 0.25, 0.1, 0.5, 0.75, 0.9, 1]: #  0.5, 0.1, 0, 0.9, 1 ]: #1, 0,  0.9, #   1000, 100,  10, 1]:
    for CrossEntropyLoss in [1]:
        for SonoData in [False]:
            for doAugmentation in [False]:
                for doPretrain in [False]:


                    train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)

                    if SonoData:
                        sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=numberOfSubsequentFrames,useSonoData=True)
                        train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)
                    else:
                        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=numberOfSubsequentFrames,useSonoData=False)
                    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = bs, num_workers = numWork, shuffle = True, pin_memory = True, drop_last = True)

                    dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                    device = torch.device("cuda")

                    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                    model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                    if device.type == "cuda":
                        model = torch.nn.DataParallel(model)
                    model.to(device)

                    if doPretrain:
                        print("load from weights")
                        checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                        model.load_state_dict(checkpoint['state_dict'])

                    #optim = torch.optim.SGD(model.parameters(), lr=1000, momentum=0.9)
                    learning_rate = 1e-2
                    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    #optim = torch.optim.Adagrad(model.parameters(), lr=100)

                    bestLoss = 1000000

                    with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_withCrossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint) + "_"+str(numEpochs)+"_Adam.csv", 'w') as output:
                        for epoch in range(numEpochs):
                            for phase in dataloaders.keys():

                                total = 0
                                n = 0
                                dsctotal = 0
                                cetotal = 0
                                constrainedTotal = 0

                                with torch.set_grad_enabled(phase != 'val'):
                                    with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                        for (i, (x,y)) in enumerate(dataloaders[phase]):

                                            x = x.to(device)
                                            y = y.to(device)
                                            #print(y.min(), y.max())
                                            #print(x.shape)

                                            #print("input shape", x.shape)

                                            # reshape to be bs x frames indepedent predictions on (3,112,112) images
                                            x = torch.transpose(x, 1, 4)
                                            #print(x.shape)
                                            x = x.reshape(bs * numberOfSubsequentFrames,  112, 112, 3).transpose(1,2)
                                            x = torch.transpose(x, 1, 3).type(torch.float)

                                            #print("new input shape", x.shape)

                                            p = model(x)["out"]

                                            #print("predictions shape", p.shape)

                                            constrainedLoss = variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames) / 112 / 112
                                            dscloss = x.shape[0] / numberOfSubsequentFrames - variousLossFunctions.diceLoss(p[0::numberOfSubsequentFrames, 0, :, :], y,)
                                            celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsequentFrames, 0, :, :], y, reduction = "sum") / 112 / 112

                                            loss =  (1-weightOfConstraint) * celoss + weightOfConstraint * constrainedLoss
                                            #CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[0::numberOfSubsquentFrames, 0, :, :], y, reduction = "sum") / 112 / 112 + variousLossFunctions.constrainSubsequentLoss(p, bs, numberOfSubsequentFrames).item()#x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,) +
                                            
#                                            loss = loss.item()
                                            total += loss#.item()
                                            dsctotal += dscloss#.item()
                                            cetotal += celoss#.item()
                                            constrainedTotal += constrainedLoss

                                            n += x.shape[0] / numberOfSubsequentFrames

                                            #print(loss, total, n, flush = True)

                                            progressbar.set_postfix_str(phase + "{:.1f} total{:.4f} dsc{:.4f} ce{:.4f} constrain{:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , constrainedTotal / n, loss / x.size(0) * numberOfSubsequentFrames )) #/ 112 / 112
                                            progressbar.update()

                                            if(phase != 'val'):
                                                optim.zero_grad()
                                                loss.backward()
                                                optim.step()

                                        output.write(phase + "," + str(epoch) + "," + str(total.item() / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + ',' + str(constrainedTotal.item() / n) + "\n") #/ 112 / 112

                                        if phase == 'val':
                                            currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) + "_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_current.pt"
                                            torch.save(model.state_dict(), currentRVweights)

                                            if (total / n ) < bestLoss: #/ 112 / 112)
                                                bestLoss = (total / n ) # / 112 / 112)
                                                print("this is the best so far", str(bestLoss), flush = True)
                                                bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_crossEntropyOnly"+ str(CrossEntropyLoss) +"_withL2ConstrainedLoss"+ str(weightOfConstraint)  + "_"+str(numEpochs)+"_Adam_best.pt"
                                                torch.save(model.state_dict(), bestRVweights)


In [ ]:
# hyperparameter search without constrained loss

#bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_SQLguide_True_AugmentTrue_best.pt"
bestLVweights = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\EchoNetDynamic-Weights\\deeplabv3_resnet50_random.pt"

for diceLoss in [0,1]:
    for CrossEntropyLoss in [0,1]:
        for SonoData in [ False, True]:
            for doAugmentation in [False, True]:
                for doPretrain in [True]:
                    if(diceLoss + CrossEntropyLoss > 0):
                        print(diceLoss, CrossEntropyLoss, SonoData, doAugmentation, doPretrain)
                        
                        train_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=0,useSonoData=False)

                        if SonoData:
                            sono_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="train", fuzzyAugmentation=doAugmentation,constrainSubsequent=0,useSonoData=True)
                            train_dataloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_dataset,sono_dataset]), batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)
                        else:
                            train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)

                        val_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="val", fuzzyAugmentation=False,constrainSubsequent=0,useSonoData=False)
                        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 100, num_workers = 5, shuffle = True, pin_memory = True, drop_last = True)

                        dataloaders = {'train':train_dataloader, 'val':val_dataloader}

                        device = torch.device("cuda")

                        model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
                        model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

                        if device.type == "cuda":
                            model = torch.nn.DataParallel(model)
                        model.to(device)

                        if doPretrain:
                            print("load from weights")
                            checkpoint = torch.load(bestLVweights)#LVWeights) #currentRVweights)#
                            model.load_state_dict(checkpoint['state_dict'])

                        learning_rate = 1e-4
                        optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

                        # optim = torch.optim.Adagrad(model.parameters(), lr=100)

                        bestLoss = 1000000

                        with open("LossAcrossEpochs_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) + "_Adam.csv", 'w') as output:
                            for epoch in range(50):
                                for phase in dataloaders.keys():

                                    total = 0
                                    n = 0
                                    dsctotal = 0
                                    cetotal = 0

                                    with torch.set_grad_enabled(phase != 'val'):
                                        with tqdm.tqdm(total = len(dataloaders[phase])) as progressbar:
                                            for (i, (x,y)) in enumerate(dataloaders[phase]):

                                                x = x.to(device)
                                                y = y.to(device)
                                                #print(y.min(), y.max())
                                                #print(x.shape)
                                                p = model(x)["out"]
                                                loss = diceLoss * (x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,)) + CrossEntropyLoss * torch.nn.functional.binary_cross_entropy_with_logits(p[:, 0, :, :], y, reduction = "sum") / 112 / 112 #+ variousLossFunctions.diceLoss(p[:, 0, :, :], y,)
                                                dscloss = x.shape[0] - variousLossFunctions.diceLoss(p[:, 0, :, :], y,)
                                                celoss = torch.nn.functional.binary_cross_entropy_with_logits(p[:, 0, :, :], y, reduction = "sum") / 112 / 112

                                                total += loss.item()
                                                dsctotal += dscloss#.item()
                                                cetotal += celoss#.item()

                                                n += x.shape[0]

                                                #print(loss, total, n, flush = True)

                                                progressbar.set_postfix_str(phase + "{:.1f} {:.4f} {:.4f} {:.4f}  ({:.4f})".format( epoch, total / n , dsctotal / n , cetotal / n , loss.item() / x.size(0) )) #/ 112 / 112
                                                progressbar.update()

                                                if(phase != 'val'):
                                                    optim.zero_grad()
                                                    loss.backward()
                                                    optim.step()

                                            output.write(phase + "," + str(epoch) + "," + str(total / n ) + "," + str(dsctotal.item() / n ) + "," + str(cetotal.item() / n ) + "\n") #/ 112 / 112

                                            if phase == 'val':
                                                currentRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) + "_Adam_current.pt"
                                                torch.save(model.state_dict(), currentRVweights)

                                                if (total / n ) < bestLoss: #/ 112 / 112)
                                                    bestLoss = (total / n ) # / 112 / 112)
                                                    print("this is the best so far", str(bestLoss), flush = True)
                                                    bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Pretrain"+str(doPretrain)+"_Sono"+str(SonoData)+"_Augment"+str(doAugmentation)+ "_diceLoss"+ str(diceLoss) + "_CrossEntropyLoss"+ str(CrossEntropyLoss) +"_Adam_best.pt"
                                                    torch.save(model.state_dict(), bestRVweights)


In [ ]:
def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i


In [ ]:
# Generate videos from both test set of RV focused Views

device = torch.device("cuda")
testFolder = "C:\\Users\\Windows\\Dropbox\\Echo Research\\CodeBase\\DynamicRV\\Constrained\\SmallTestSet"


dataloader = torch.utils.data.DataLoader(echonet.datasets.Echo(split="external_test", external_test_location = testFolder, target_type=["Filename"], length=None, period=1, mean=mean, std=std),
                                        batch_size=100, num_workers=0, shuffle=False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)


# test_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="test", fuzzyAugmentation=False,constrainSubsequent=0,useSonoData=False)
# dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, num_workers = 0, shuffle = False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)
          
stem = "C:\\Datasets\\RVWork\\RV_deeplab_50_PretrainFalse_SonoFalse_AugmentFalse_crossEntropyOnly1_with"
# foldername = ["FullFeatures_DiceLoss_RVViews","FullFeatures_CELoss_RVViews","FullFeatures_BothLoss_RVViews"]
loss = ["L2", "L1"]
weight = ["1", "0.9", "0.75", "0.5", "0.25", "0.1", "0"]
folder = "Constrained\\ExamplesWithBiggerLoss"
bestRVweights = ["ConstrainedLoss1_50_Adam_best.pt",
                "ConstrainedLoss0.9_50_Adam_best.pt",
                 "ConstrainedLoss0.75_50_Adam_best.pt",
                 "ConstrainedLoss0.5_50_Adam_best.pt",
                 "ConstrainedLoss0.25_50_Adam_best.pt",
                 "ConstrainedLoss0.1_50_Adam_best.pt",
                 "ConstrainedLoss0_50_Adam_best.pt"]


model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size = model.classifier[-1].kernel_size)

if device.type == "cuda":
     model = torch.nn.DataParallel(model)
model.to(device)

for thisLoss in loss:
    for count, thisweight in enumerate(bestRVweights):
        print(count, thisLoss, thisweight)
        checkpoint = torch.load(stem + thisLoss + thisweight)
        model.load_state_dict(checkpoint)


        #all_dataset = dataset.Dataset(SQLtracings, SQLvideos, mean, std, split="all", fuzzyAugmentation=False,constrainSubsequent=0)
        #dataloader = torch.utils.data.DataLoader(all_dataset, batch_size = 200, num_workers = 10, shuffle = True, pin_memory = True, drop_last = True)

        #bestRVweights = "C:\\Datasets\RVWork\\RV_deeplab_50_Sono_True_Pretrain_False_Augment_False_best.pt"
        #
        #checkpoint = torch.load(bestRVweights)
        #model.load_state_dict(checkpoint['state_dict'])

        #os.makedirs(folder, exist_ok=True)

        block = 256
        with torch.no_grad():
            for (x, f, i) in tqdm.tqdm(dataloader):
                x = x.to(device)
                y = np.concatenate([model(x[i:(i + block), :, :, :])["out"].detach().cpu().numpy() for i in range(0, x.shape[0], block)]).astype(np.float16)

                start = 0
                x = (x.cpu().numpy() * std.reshape((1, 3, 1, 1))) + mean.reshape((1, 3, 1, 1))

                for (filename, offset) in zip(f, i):
                    img = x[start:(start + offset), ...].transpose((1, 0, 2, 3))
                    img = np.concatenate((img, img), 3)
                    logit = y[start:(start + offset), 0, :, :].copy()
                    img[1, :, :, 112:] = np.maximum(255. * (logit > 0), img[0, :, :, 112:])

                    #img = y[start:(start + offset)]
                    #np.save(os.path.join(output, "labels", os.path.splitext(filename)[0]), y[start:(start + offset), 0, :, :])
                    echonet.utils.savevideo(os.path.join(folder, os.path.splitext(filename)[0] + "_" + thisLoss + "_"+ weight[count] + ".avi"), img.astype(np.uint8), 50)

                    start += offset